In [1]:
import pandas as pd
import pymongo
import uuid

In [28]:
def get_books_dataframe():
    df = pd.read_csv(
        "books_data/books.csv",
        sep=";",
        on_bad_lines="skip",
        encoding="latin-1",
    )

    return df[["ISBN", "Book-Title", "Book-Author", "Year-Of-Publication"]].rename(
        columns={
            "ISBN": "isbn",
            "Book-Title": "title",
            "Book-Author": "author",
            "Year-Of-Publication": "year",
        }
    )


def get_books_authors_relations(df):
    authors_map = {author: uuid.uuid4().hex for author in df["author"].unique().tolist()}
    author_book = df[["author", "isbn"]]
    author_book["author_id"] = author_book["author"].map(authors_map)

    books = df[["isbn", "title", "year"]].to_dict(orient="records")
    authors = (
        author_book[["author_id", "author"]]
        .drop_duplicates()
        .rename(columns={"author": "name"})
        .to_dict(orient="records")
    )
    authors_book = author_book[["author_id", "isbn"]].to_dict(orient="records")

    return books, authors, authors_book

# Mongo DB

In [17]:
my_client = pymongo.MongoClient("mongodb://localhost:27017/")

In [18]:
db = my_client["books_db"]

## Query elements

In [21]:
def insert_from_dataframe(df, db):
    books, authors, authors_book = get_books_authors_relations(df)

    db.books.insert_many(books)
    db.authors.insert_many(authors)
    db.authors_book.insert_many(authors_book)


df = get_books_dataframe()
insert_from_dataframe(df[:100], db)

In [25]:
author_1 = db.authors.find_one({"name": "Mark P. O. Morford"})
author_2 = db.authors.find_one({"name": "Richard Bruce Wright"})

db.authors.update_one({"_id": author_1["_id"]}, {"$set": {"name": author_1["name"].upper()}})
db.authors.update_one({"_id": author_2["_id"]}, {"$set": {"name": author_2["name"].upper()}})

db.authors.find_one({"name": "MARK P. O. MORFORD"})

{'_id': ObjectId('654575fff98b4d393b98e18f'),
 'author_id': '02c79343d4274fe3a82f326f9d2a271f',
 'name': 'MARK P. O. MORFORD'}

In [20]:
def queary_book():
    author_name = "DAVID FRUM"
    year = 2020
    prefix = "The"

    author_id = db.authors.find_one({"name": author_name})["author_id"]
    db.authors_book.aggregate(
        [
            {
                "$lookup": {
                    "from": "books",
                    "localField": "isbn",
                    "foreignField": "isbn",
                    "as": "book",
                }
            },
            {
                "$match": {
                    "author_id": author_id,
                    "book.year": {"$gt": year},
                    "book.title": {"$regex": rf"^{prefix}"},
                }
            },
        ]
    )

%timeit queary_book() 

1.41 ms ± 95.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [19]:
%timeit db.books.find_one({"isbn": "0195153448"})

421 µs ± 19.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [20]:
db.books.delete_many({})
db.authors.delete_many({})
db.authors_book.delete_many({})

DeleteResult({'n': 100, 'ok': 1.0}, acknowledged=True)

# Neo4j

In [2]:
df = get_books_dataframe()

NameError: name 'get_books_dataframe' is not defined

In [4]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
user = "neo4j"
password = "Deusto123"

driver = GraphDatabase.driver(uri, auth=(user, password))

In [3]:
def create_book(tx, isbn, year, title):
    query = r"CREATE (b:Book {isbn: $isbn, year: $year, title: $title})"
    tx.run(query, isbn=isbn, year=year, title=title)


def create_author(tx, author_id, name):
    query = r"CREATE (a:Author {id: $author_id, name: $name})"
    tx.run(query, author_id=author_id, name=name)


def create_relationship(tx, isbn, author_id):
    query = r"""
    MATCH (b:Book), (a:Author)
    WHERE b.isbn = $isbn AND a.id = $author_id
    CREATE (b)-[:ESCRITO_POR]->(a)
    """
    tx.run(query, isbn=isbn, author_id=author_id)


def update_author_name_to_uppercase(tx, name):
    query = r"""
    MATCH (a:Author)
    WHERE a.name = $name
    SET a.name = toUPPER(a.name)
    """
    tx.run(query, name=name)


def get_book_by_author_name(tx, name, year, prefix):
    query = r"""
    MATCH p=(b:Book)-[r:ESCRITO_POR]->(a:Author)
    WHERE b.title STARTS WITH $prefix AND b.year > $year 
    AND a.name = $name
    RETURN b 
    """
    result = tx.run(query, name=name, year=year, prefix=prefix)
    return [r.data()["b"] for r in result]


def get_book_by_isbn(tx, isbn):
    query = r"""
    MATCH (b: Book)
    WHERE b.isbn = $isbn
    RETURN b
    """
    result = tx.run(query, isbn=isbn)
    return [r.data()["b"] for r in result]


def delete_all_data(tx):
    query = "MATCH (n) DETACH DELETE n"
    tx.run(query)

In [59]:
books, authors, authors_book = get_books_authors_relations(df[:100])

with driver.session() as session:
    session.execute_write(delete_all_data)

    for book in books:
        session.execute_write(create_book, **book)

    for author in authors:
        session.execute_write(create_author, **author)

    for author_book in authors_book:
        session.execute_write(create_relationship, **author_book)

/tmp/ipykernel_4038/1119965467.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  author_book["author_id"] = author_book["author"].map(authors_map)


In [6]:
def make_update():
    with driver.session() as session:
        session.execute_write(update_author_name_to_uppercase, name="Mark P. O. Morford")
        session.execute_write(update_author_name_to_uppercase, name="Richard Bruce Wright")

%timeit make_update() 

13.1 ms ± 1.54 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
def make_query():
    with driver.session() as session:
        author_name = "DAVID FRUM"
        year = 2001
        prefix = "The"

        book = session.execute_read(get_book_by_isbn, "0375509038")
        # value = session.execute_read(get_book_by_author_name, author_name, year, prefix)
%timeit make_query() 

2.66 ms ± 59.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
